除了GRAPHS栏目外，tensorboard还有IMAGES、AUDIO、SCALARS、HISTOGRAMS、DISTRIBUTIONS、FROJECTOR、TEXT、PR CURVES、PROFILE九个栏目，本小节将详细介绍这些子栏目各自的特点和用法。

[ Tensorflow监控指标可视化](https://blog.csdn.net/fendouaini/article/details/80368770)

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
tf.reset_default_graph()

#载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

#每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)#平均值, 记录存储值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)#标准差
        tf.summary.scalar('max', tf.reduce_max(var))#最大值
        tf.summary.scalar('min', tf.reduce_min(var))#最小值
        tf.summary.histogram('histogram', var)#直方图，用于生成分布图

#命名空间
with tf.name_scope('input'):
    #定义两个placeholder
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    y = tf.placeholder(tf.float32,[None,10],name='y-input')
    # Tensorboard  images
    image_shape=tf.reshape(x, [-1, 28, 28,1])
    tf.summary.image('input', image_shape, 10)
    
with tf.name_scope('layer'):
    #创建一个简单的神经网络
    with tf.name_scope('wights'):
        W = tf.Variable(tf.zeros([784,10]),name='W')
        variable_summaries(W)
    with tf.name_scope('biases'):    
        b = tf.Variable(tf.zeros([10]),name='b')
        variable_summaries(b)
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x,W) + b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b)
        #tf.summary.pr_curve(name='foo', predictions=prediction, labels=y, num_thresholds=10) 


#二次代价函数
# loss = tf.reduce_mean(tf.square(y-prediction))
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)
    
with tf.name_scope('train'):
    #使用梯度下降法
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        #结果存放在一个布尔型列表中
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大的值所在的位置
    with tf.name_scope('accuracy'):
        #求准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
        
#合并所有的summary
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/',sess.graph)
    for epoch in range(50):
        for batch in range(n_batch):
            batch_xs,batch_ys =  mnist.train.next_batch(batch_size)
            summary,_ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys})
            
        writer.add_summary(summary,epoch)
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter " + str(epoch) + ",Testing Accuracy " + str(acc))
        

W0722 15:36:25.659719 11172 deprecation.py:323] From <ipython-input-2-474dac0c4e2e>:4: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0722 15:36:25.665704 11172 deprecation.py:323] From D:\Program Files (x86)\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0722 15:36:25.667698 11172 deprecation.py:323] From D:\Program Files (x86)\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for 

Extracting MNIST_data\train-images-idx3-ubyte.gz


W0722 15:36:25.996862 11172 deprecation.py:323] From D:\Program Files (x86)\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0722 15:36:26.006799 11172 deprecation.py:323] From D:\Program Files (x86)\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0722 15:36:26.081745 11172 deprecation.py:323] From D:\Program Files (x86)\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be 

Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


W0722 15:36:26.446769 11172 deprecation.py:323] From <ipython-input-2-474dac0c4e2e>:50: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Iter 0,Testing Accuracy 0.8238
Iter 1,Testing Accuracy 0.8864
Iter 2,Testing Accuracy 0.9013
Iter 3,Testing Accuracy 0.9052
Iter 4,Testing Accuracy 0.9078
Iter 5,Testing Accuracy 0.9098
Iter 6,Testing Accuracy 0.9109
Iter 7,Testing Accuracy 0.9135
Iter 8,Testing Accuracy 0.9142
Iter 9,Testing Accuracy 0.9163
Iter 10,Testing Accuracy 0.9165
Iter 11,Testing Accuracy 0.9186
Iter 12,Testing Accuracy 0.9188
Iter 13,Testing Accuracy 0.9197
Iter 14,Testing Accuracy 0.9185
Iter 15,Testing Accuracy 0.9188
Iter 16,Testing Accuracy 0.9214
Iter 17,Testing Accuracy 0.9205
Iter 18,Testing Accuracy 0.9214
Iter 19,Testing Accuracy 0.9203
Iter 20,Testing Accuracy 0.9221
Iter 21,Testing Accuracy 0.9216
Iter 22,Testing Accuracy 0.9218
Iter 23,Testing Accuracy 0.9228
Iter 24,Testing Accuracy 0.9232
Iter 25,Testing Accuracy 0.9233
Iter 26,Testing Accuracy 0.923
Iter 27,Testing Accuracy 0.9238
Iter 28,Testing Accuracy 0.9232
Iter 29,Testing Accuracy 0.9241
Iter 30,Testing Accuracy 0.9239
Iter 31,Testing Acc

In [3]:
# for i in range(2001):
#     #m每个批次100个样本
#     batch_xs,batch_ys = mnist.train.next_batch(100)
#     summary,_ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys})
#     writer.add_summary(summary,i)
#     if i%500 == 0:
#         print(sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels}))